### LangGraph Agent - Customer Support multivoice Agent

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

system = """You are Andrea, a knowledgeable and friendly assistant in a telecommunications company. Your expertise lies in various mobile plans and upgrades. Your role is to help users understand their options and assist them with their queries about mobile plans and upgrades. Always respond in a helpful and professional manner.
Always speak to the user with his name.

Username: {username}
UserID: {user_id}

Remember, your goal is to make the user feel supported and informed. Always be courteous and clear in your responses.
"""
prompt_template = PromptTemplate.from_template(system)

In [3]:
import requests
from langchain_core.tools import tool
from datetime import datetime, timedelta

API_URL = "http://127.0.0.1:8000"
ADMIN_USERNAME = "admin1"
ADMIN_PASSWORD = "admin1password"

@tool
def create_contract_tool(user_id: int, category: str):
    """
    Create a new contract for a user with a specific category.

    Args:
        user_id (int): ID of the user for whom the contract is being created.
        category (str): Category of the contract. Must be one of "basic", "normal", or "premium".

    Returns:
        str: A string containing the details of the created contract.
        str: Error message if the user is not found or if any other validation fails.

    This function interacts with the FastAPI API to create a new contract for the specified user.
    """
    try:
        # Login as admin
        login_response = requests.post(f"{API_URL}/token/", data={"username": ADMIN_USERNAME, "password": ADMIN_PASSWORD})
        if login_response.status_code != 200:
            return f"Login failed: {login_response.json().get('detail')}"

        access_token = login_response.json().get("access_token")
        headers = {"Authorization": f"Bearer {access_token}"}

        # Validate the category
        if category not in ["basic", "normal", "premium"]:
            return "Category must be one of: basic, normal, premium"

        # Create contract
        contract_data = {"user_id": user_id, "category": category}
        create_response = requests.post(f"{API_URL}/contracts/", json=contract_data, headers=headers)
        if create_response.status_code != 200:
            return f"Failed to create contract: {create_response.json().get('detail')}"

        contract = create_response.json()
        return f"Contract created: User ID {contract['user_id']}, Category {contract['category']}, Contract Time {contract['contract_time']}"
    except Exception as e:
        return f"Failed to execute. Error: {repr(e)}"


@tool
def update_contract_status_tool(contract_id: int, category: Optional[str] = None):
    """
    Update the status of a contract.

    Args:
        contract_id (int): ID of the contract to be updated.
        category (str, optional): New category for the contract. Must be one of "basic", "normal", or "premium".

    Returns:
        str: A string containing the details of the updated contract.
        str: Error message if the contract is not found or if any other validation fails.

    This function interacts with the FastAPI API to update the status of the specified contract.
    """
    try:
        # Login as admin
        login_response = requests.post(f"{API_URL}/token/", data={"username": ADMIN_USERNAME, "password": ADMIN_PASSWORD})
        if login_response.status_code != 200:
            return f"Login failed: {login_response.json().get('detail')}"

        access_token = login_response.json().get("access_token")
        headers = {"Authorization": f"Bearer {access_token}"}

        # Validate the category if provided
        if category and category not in ["basic", "normal", "premium"]:
            return "Category must be one of: basic, normal, premium"

        # Update contract
        update_data = {"category": category}
        update_response = requests.put(f"{API_URL}/contracts/{contract_id}/", json=update_data, headers=headers)
        if update_response.status_code != 200:
            return f"Failed to update contract: {update_response.json().get('detail')}"

        contract = update_response.json()
        return f"Contract updated: ID {contract['id']}, Category {contract['category']}, Contract Time {contract['contract_time']}"
    except Exception as e:
        return f"Failed to execute. Error: {repr(e)}"


@tool
def delete_contract_tool(contract_id: int):
    """
    Delete a contract (set it to expire in 3 months).

    Args:
        contract_id (int): ID of the contract to be deleted.

    Returns:
        str: A string confirming the deletion of the contract along with the exact date it will expire.
        str: Error message if the contract is not found or if any other validation fails.

    This function interacts with the FastAPI API to delete the specified contract.
    """
    try:
        # Login as admin
        login_response = requests.post(f"{API_URL}/token/", data={"username": ADMIN_USERNAME, "password": ADMIN_PASSWORD})
        if login_response.status_code != 200:
            return f"Login failed: {login_response.json().get('detail')}"

        access_token = login_response.json().get("access_token")
        headers = {"Authorization": f"Bearer {access_token}"}

        # Delete contract
        delete_response = requests.delete(f"{API_URL}/contracts/{contract_id}/", headers=headers)
        if delete_response.status_code != 200:
            return f"Failed to delete contract: {delete_response.json().get('detail')}"

        expiration_date = (datetime.utcnow() + timedelta(days=90)).date()
        return f"Contract will be cancelled on {expiration_date}"
    except Exception as e:
        return f"Failed to execute. Error: {repr(e)}"


@tool
def get_contract_details_tool(contract_id: int):
    """
    Get the details of a contract.

    Args:
        contract_id (int): ID of the contract to retrieve.

    Returns:
        str: A string containing the details of the contract.
        str: Error message if the contract is not found or if any other validation fails.

    This function interacts with the FastAPI API to retrieve the specified contract's details.
    """
    try:
        # Login as admin
        login_response = requests.post(f"{API_URL}/token/", data={"username": ADMIN_USERNAME, "password": ADMIN_PASSWORD})
        if login_response.status_code != 200:
            return f"Login failed: {login_response.json().get('detail')}"

        access_token = login_response.json().get("access_token")
        headers = {"Authorization": f"Bearer {access_token}"}

        # Get contract details
        get_response = requests.get(f"{API_URL}/contracts/{contract_id}/", headers=headers)
        if get_response.status_code != 200:
            return f"Failed to get contract details: {get_response.json().get('detail')}"

        contract = get_response.json()
        return f"Contract details: ID {contract['id']}, Category {contract['category']}, Contract Time {contract['contract_time']}, User ID {contract['user_id']}"
    except Exception as e:
        return f"Failed to execute. Error: {repr(e)}"


tools = [create_contract_tool, update_contract_status_tool, delete_contract_tool, get_contract_details_tool]


In [11]:
from langchain_core.messages.human import HumanMessage
from langchain_core.messages.system import SystemMessage

sys_msg = [SystemMessage(content=prompt_template.format(username="Max", user_id=4))]
hu_msg = [HumanMessage(content="I would like to create a premium contract please.")]

chat_history = []

messages = sys_msg + chat_history + hu_msg
model = ChatOpenAI()
model_with_tools = model.bind_tools(tools=tools)

result = model_with_tools.invoke(messages)

In [5]:
messages.append(result)

In [6]:
result.tool_calls

[{'name': 'create_contract_tool',
  'args': {'user_id': 4, 'category': 'premium'},
  'id': 'call_FClZJxmm8W5DDae78HRWyWD4',
  'type': 'tool_call'}]

In [7]:
from langchain_core.messages import ToolMessage

for tool_call in result.tool_calls:
    print("Use Tool:", tool_call)
    selected_tool = tool_mapping = {"create_contract_tool": create_contract_tool}[
        tool_call["name"].lower()
    ]
    tool_output = selected_tool.invoke(tool_call["args"])
    print(tool_output)
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

Use Tool: {'name': 'create_contract_tool', 'args': {'user_id': 4, 'category': 'premium'}, 'id': 'call_FClZJxmm8W5DDae78HRWyWD4', 'type': 'tool_call'}
Failed to execute. Error: ConnectionError(MaxRetryError("HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /token/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001EB7666BF50>: Failed to establish a new connection: [WinError 10061] Es konnte keine Verbindung hergestellt werden, da der Zielcomputer die Verbindung verweigerte'))"))


In [8]:
messages

[SystemMessage(content='You are Andrea, a knowledgeable and friendly assistant in a telecommunications company. Your expertise lies in various mobile plans and upgrades. Your role is to help users understand their options and assist them with their queries about mobile plans and upgrades. Always respond in a helpful and professional manner.\nAlways speak to the user with his name.\n\nUsername: Max\nUserID: 4\n\nRemember, your goal is to make the user feel supported and informed. Always be courteous and clear in your responses.\n'),
 HumanMessage(content='I would like to create a premium contract please.'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_FClZJxmm8W5DDae78HRWyWD4', 'function': {'arguments': '{"user_id":4,"category":"premium"}', 'name': 'create_contract_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 262, 'total_tokens': 282}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, '

In [9]:
model_with_tools.invoke(messages)

AIMessage(content='I apologize for the inconvenience. It seems there was an issue with creating the premium contract at the moment. Let me try to create the contract again for you.', additional_kwargs={'tool_calls': [{'id': 'call_izHGuQci9WesseRarVPm8Evn', 'function': {'arguments': '{"user_id":4,"category":"premium"}', 'name': 'create_contract_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 394, 'total_tokens': 447}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-50d17686-0436-4583-af64-61c0df653c1a-0', tool_calls=[{'name': 'create_contract_tool', 'args': {'user_id': 4, 'category': 'premium'}, 'id': 'call_izHGuQci9WesseRarVPm8Evn', 'type': 'tool_call'}], usage_metadata={'input_tokens': 394, 'output_tokens': 53, 'total_tokens': 447})

In [ ]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from typing_extensions import TypedDict

from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition


class State(TypedDict):
    messages: Annotated[list, add_messages]
    ask_human: bool
    agent_name: str

In [ ]:
from langchain_core.tools import tool


@tool
def request_assistance():
    """Escalate the conversation to an expert. Use this if you are unable to assist directly or if the user requires support beyond your permissions.

    To use this function, relay the user's 'request' so the expert can provide the right guidance.
    """
    return ""